# 1 Introduction
At Santander their mission is to help people and businesses prosper. they are always looking for ways to help our customers understand their financial health and identify which products and services might help them achieve their monetary goals.

# 2.0 Load packages

# 2.1 Import

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool
from IPython.display import display
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import norm
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import time
import glob
import sys
import os
import gc

# 2.2 Setup

In [8]:
# for get better results change fold_n to 5
fold_n = 5
folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=10)
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)
pd.set_option('display.max_columns', 200)

# 2.3 Version

In [9]:
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('Python: {}'.format(sys.version))

pandas: 0.22.0
numpy: 1.16.0
Python: 3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]


# 3.0 Problem Definition
In this challenge, we should help this bank identify which customers will make a specific transaction in the future, irrespective of the amount of money transacted. The data provided for this competition has the same structure as the real data we have available to solve this problem.

# 3.1 Problem Feature
* train.csv - the training set.
* test.csv - the test set. The test set contains some rows which are not included in scoring.
* sample_submission.csv - a sample submission file in the correct format.

# 3.2 Aim
In this competition, The task is to predict the value of target column in the test set.

# 3.3 Variables
We are provided with an anonymized dataset containing numeric feature variables, the binary target column, and a string ID_code column.

The task is to predict the value of target column in the test set.

# 3.4 evaluation
Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

In [11]:
from sklearn.metrics import roc_auc_score, roc_curve

# 4.0 Exploratory Data Analysis(EDA)
In this section, we'll analysis how to use graphical and numerical techniques to begin uncovering the structure of your data.

Data Collection
Visualization
Data Preprocessing
Data Cleaning

In [12]:
# import Datasets to play with it
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [13]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission.head()

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [14]:
train.shape, test.shape, sample_submission.shape

((200000, 202), (200000, 201), (200000, 2))

In [15]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.925500000000000,-6.7863,11.908099999999999,5.093000000000000,11.460699999999999,-9.2834,5.1187,18.626600000000000,-4.9200,5.747000000000001,2.9252,3.182100000000000,14.013700000000000,0.574500000000000,8.7989,14.569100000000001,5.748700000000000,-7.239300000000000,4.284000000000000,30.713300000000000,10.535000000000000,16.219100000000001,2.579100000000000,2.4716,14.383100000000001,13.432499999999999,-5.1488,-0.4073,4.9306,5.9965,-0.308500000000000,12.904100000000000,-3.8766,16.891100000000002,11.192000000000000,10.5785,0.6764,7.8871,4.666700000000000,3.8743,-5.238700000000000,7.3746,11.576700000000001,12.044600000000001,11.641800000000000,-7.017000000000000,5.922600000000000,-14.213600000000000,16.028300000000002,5.325300000000000,12.919400000000000,29.045999999999999,-0.6940,5.1736,-0.747400000000000,14.832200000000000,11.266800000000000,5.3822,2.0183,10.116600000000000,16.182800000000000,4.959000000000000,2.0771,-0.2154,8.674799999999999,9.5319,5.805600000000000,22.432099999999998,5.0109,-4.701000000000001,21.637400000000000,0.5663,5.1999,8.859999999999999,43.112699999999997,18.381599999999999,-2.3440,23.410399999999999,6.5199,12.198300000000000,13.646800000000001,13.837199999999999,1.367500000000000,2.942300000000000,-4.5213,21.466899999999999,9.3225,16.459700000000002,7.998400000000000,-1.7069,-21.449400000000001,6.7806,11.092400000000000,9.991300000000001,14.842100000000000,0.1812,8.964200000000000,16.257200000000001,...,9.476300000000000,13.310200000000000,26.537600000000001,1.4403,14.710000000000001,6.0454,9.542600000000000,17.155400000000000,14.110400000000000,24.362700000000000,2.0323,6.7602,3.9141,-0.4851,2.5240,1.5093,2.5516,15.575200000000001,-13.422100000000000,7.2739,16.009399999999999,9.726800000000001,0.8897,0.775400000000000,4.2218,12.003900000000000,13.857100000000001,-0.7338,-1.9245,15.446199999999999,12.828700000000000,0.3587,9.650800000000000,6.5674,5.1726,3.134500000000000,29.454699999999999,31.404499999999999,2.8279,15.659900000000000,8.3307,-5.601100000000000,19.061399999999999,11.266299999999999,8.698900000000000,8.369400000000001,11.565899999999999,-16.472700000000000,4.0288,17.924399999999999,18.517700000000001,10.779999999999999,9.005599999999999,16.696400000000001,10.4838,1.657300000000000,12.174899999999999,-13.132400000000001,17.605399999999999,11.542299999999999,15.457599999999999,5.3133,3.6159,5.038400000000000,6.676000000000000,12.664400000000001,2.7004,-0.6975,9.598100000000001,5.4879,-4.7645,-8.4254,20.87730000

Reducing memory size by ~50%
Because we make a lot of calculations in this kernel, we'd better reduce the size of the data.

* 300 MB before Reducing
* 150 MB after Reducing

In [16]:
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

Reducing for train and test

In [17]:
train, NAlist = reduce_mem_usage(train)
print("-"*50)
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("-"*50)
print(NAlist)

Memory usage of properties dataframe is : 308.2276153564453  MB
******************************
Column:  target
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  var_0
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_1
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_2
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_3
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_4
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_5
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_6
dtype before:  float64
dtype after:  fl

dtype after:  float32
******************************
******************************
Column:  var_66
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_67
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_68
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_69
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_70
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_71
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_72
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_73
dtype before:  float64
dtype after:  fl

dtype after:  float32
******************************
******************************
Column:  var_134
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_135
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_136
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_137
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_138
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_139
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_140
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_141
dtype before:  float64
dtype af

In [18]:
test, NAlist = reduce_mem_usage(test)
print("-"*50)
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("-"*50)
print(NAlist)

Memory usage of properties dataframe is : 306.7017364501953  MB
******************************
Column:  var_0
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_1
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_2
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_3
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_4
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_5
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_6
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_7
dtype before:  float64
dtype after: 

dtype after:  float32
******************************
******************************
Column:  var_67
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_68
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_69
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_70
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_71
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_72
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_73
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_74
dtype before:  float64
dtype after:  fl

dtype after:  float32
******************************
******************************
Column:  var_135
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_136
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_137
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_138
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_139
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_140
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_141
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  var_142
dtype before:  float64
dtype af

# 4.2 Data set fields

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float32(200), object(1), uint8(1)
memory usage: 154.3+ MB


# 4.3 Numerical values Describe

In [22]:
train.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,...,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.000000000000000,200000.0000

# 4.4 Visualizations

Visualizations done in the baseline script.

# 4.5 Data Preprocessing

In [27]:
def check_missing_data(df):
    flag=df.isna().sum().any()
    if flag==True:
        total = df.isnull().sum()
        percent = (df.isnull().sum())/(df.shape[0])
        output = pd.concat([total,percent], axis=1, keys=['Total','Percent'])
        
        data_type = []
        for col in df.columns:
            dtype = str(df[col].dtype)
            data_type.append(dtype)
        output['Types'] = data_type
        return (np.transpose(output))
    else:
        return(False)

In [28]:
check_missing_data(train)

False

In [29]:
check_missing_data(test)

False

# 4.6 Binary Classification

In [30]:
train['target'].unique()

array([0, 1], dtype=uint64)

# 4.7 Is data set imblance?

A large part of the data is unbalanced, but how can we solve it?

In [32]:
train['target'].value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [35]:
def check_balance(df, target):
    check=[]
    print('size of data is : ', df.shape[0])
    for i in [0,1]:
        print('for target {} ='.format(i))
        print(df[target].value_counts()[i]/df.shape[0]*100, '%')

In [36]:
check_balance(train, 'target')

size of data is :  200000
for target 0 =
89.95100000000001 %
for target 1 =
10.049 %


# 4.8 skewnees and kurtosis

In [38]:
print('Skewness : ', train['target'].skew())
print('Kurtosis : ', train['target'].kurt())

Skewness :  2.6576420477382454
Kurtosis :  5.063111884925181


# 5.0 Machine learning Explainability for Santander

In this section, I want to try extract insights from models with the help of this excellent Course in Kaggle. The Goal behind of ML Explainability for Santander is:

* All features are senseless named.(var_1, var2,...) but certainly the importance of each one is different!
* Extract insights from models.
* Find the most inmortant feature in models.
* Affect of each feature on the model's predictions.

# 5.1 Permutation Importance

In this section we will answer following question:
* What features have the biggest impact on predictions.
* How to extract insights form models

Prepare our data for modeling

In [39]:
cols = ['target', 'ID_code']
X = train.drop(cols, axis=1)
y = train['target']
X_test = test.drop('ID_code', axis=1)

Create a sample model to calculate which feature and more important

In [40]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
rfc_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

# 5.2 How to calculate and show importances?

Here is how to calculate and show importances with the eli5 library:

In [43]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rfc_model, random_state=1).fit(val_X, val_y)

In [44]:
eli5.show_weights(perm, feature_names = val_X.columns.tolist(), top=150)

Weight,Feature
0.0002 ± 0.0002,var_110
0.0001 ± 0.0001,var_157
0.0001 ± 0.0001,var_162
0.0001 ± 0.0001,var_42
0.0001 ± 0.0002,var_170
0.0001 ± 0.0002,var_174
0.0001 ± 0.0001,var_188
0.0001 ± 0.0001,var_147
0.0001 ± 0.0001,var_197
0.0001 ± 0.0001,var_47


# 5.3 What can be inferred from the above?
* As you move down the top of the graph, the importance of the feature decreases.
* The features that are shown in green indicate that they have a positive impact on our prediction
* The features that are shown in white indicate that they have no effect on our prediction
* The features shown in red indicate that they have a negative impact on our prediction
* The most important feature was Var_110.

# 5.4 Partial Dependence Plots

While feature importance shows what variables most affect predictions, partial dependence plots show how feature affects predictions. Partial plots dependence plots are calculated after a model has been fit.

In [45]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
tree_model = DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_split=5).fit(train_X, train_y)

For the sake of explanation, i use Decision Tree which you can see below.

In [48]:
features = [c for c in train.columns if c not in ['ID_code', 'target']]

In [49]:
from sklearn import tree
import graphviz
tree_graph = tree.export_graphviz(tree_model, out_file=None, feature_names=features)

# 5.5 Partial Dependence Plot
In this section, we see the impact of the main variables discovered in the previous sections by using the pdpbox

In [52]:
from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots

ModuleNotFoundError: No module named 'pdpbox'

# 6.0 Model Development
So far, we have used two models, and at this point we add another model and we'll be expanding it soon. In this section you will see the following model:

* lightgbm
* RandomForestClassifier
* DecisionTreeClassifier
* CatBoostClassifier

# 6.1 Lightgbm

In [53]:
# params is based on following kernel https://www.kaggle.com/brandenkmurray/nothing-works
params = {'objective' : "binary", 
               'boost':"gbdt",
               'metric':"auc",
               'boost_from_average':"false",
               'num_threads':8,
               'learning_rate' : 0.01,
               'num_leaves' : 13,
               'max_depth':-1,
               'tree_learner' : "serial",
               'feature_fraction' : 0.05,
               'bagging_freq' : 5,
               'bagging_fraction' : 0.4,
               'min_data_in_leaf' : 80,
               'min_sum_hessian_in_leaf' : 10.0,
               'verbosity' : 1}

In [54]:
%%time
y_pred_lgb = np.zeros(len(X_test))
num_round = 1000000
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
    
    lgb_model = lgb.train(params, train_data, num_round, 
                          valid_sets = [train_data, valid_data], 
                          verbose_eval = 100, early_stopping_rounds = 3500)
    
    y_pred_lgb += lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration) / 5

Fold 0 started at Mon Mar 11 17:23:34 2019
Training until validation scores don't improve for 3500 rounds.
[100]	training's auc: 0.866351	valid_1's auc: 0.848317
[200]	training's auc: 0.880308	valid_1's auc: 0.862116
[300]	training's auc: 0.882352	valid_1's auc: 0.863343
[400]	training's auc: 0.887447	valid_1's auc: 0.867385
[500]	training's auc: 0.89051	valid_1's auc: 0.870442
[600]	training's auc: 0.892927	valid_1's auc: 0.872071
[700]	training's auc: 0.894784	valid_1's auc: 0.873462
[800]	training's auc: 0.897151	valid_1's auc: 0.875473
[900]	training's auc: 0.899576	valid_1's auc: 0.877511
[1000]	training's auc: 0.901253	valid_1's auc: 0.878517
[1100]	training's auc: 0.902566	valid_1's auc: 0.879466
[1200]	training's auc: 0.904032	valid_1's auc: 0.88051
[1300]	training's auc: 0.905393	valid_1's auc: 0.88143
[1400]	training's auc: 0.906798	valid_1's auc: 0.882518
[1500]	training's auc: 0.90827	valid_1's auc: 0.883536
[1600]	training's auc: 0.90941	valid_1's auc: 0.884121
[1700]	trai

[14600]	training's auc: 0.967	valid_1's auc: 0.896169
[14700]	training's auc: 0.967276	valid_1's auc: 0.896175
[14800]	training's auc: 0.967563	valid_1's auc: 0.896132
Early stopping, best iteration is:
[11318]	training's auc: 0.957332	valid_1's auc: 0.896572
Fold 1 started at Mon Mar 11 17:32:44 2019
Training until validation scores don't improve for 3500 rounds.
[100]	training's auc: 0.866077	valid_1's auc: 0.853287
[200]	training's auc: 0.879202	valid_1's auc: 0.868413
[300]	training's auc: 0.880905	valid_1's auc: 0.868771
[400]	training's auc: 0.886127	valid_1's auc: 0.874196
[500]	training's auc: 0.888773	valid_1's auc: 0.876785
[600]	training's auc: 0.891173	valid_1's auc: 0.879111
[700]	training's auc: 0.8933	valid_1's auc: 0.880299
[800]	training's auc: 0.895925	valid_1's auc: 0.882355
[900]	training's auc: 0.898492	valid_1's auc: 0.884487
[1000]	training's auc: 0.900252	valid_1's auc: 0.885857
[1100]	training's auc: 0.901356	valid_1's auc: 0.886766
[1200]	training's auc: 0.902

KeyboardInterrupt: 

# 6.2 RandomForestClassifier

In [ ]:
y_pred_rfc = rfc_model.predict(X_test)

# 6.3 DecisionTreeClassifier

In [ ]:
y_pred_tree = tree_model.predict(X_test)

# 6.4 CatBoostClassifier

In [ ]:
train_pool = Pool(train_X, train_y)
cat_model = CatBoostClassifier(iterations=3000, learning_rate=0.03,
                              objective='Logloss',
                              eval_metric='AUC')
cat_model.fit(train_X, train_y, silent=True)
y_pred_cat = cat_model.predict(X_test)

Now you can change your model and submit the results of other models.

In [ ]:
submission_rfc = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": y_pred_rfc
    })
submission_rfc.to_csv('submission_rfc.csv', index=False)

In [ ]:
submission_tree = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": y_pred_tree
    })
submission_tree.to_csv('submission_tree.csv', index=False)

In [ ]:
submission_cat = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": y_pred_cat
    })
submission_cat.to_csv('submission_cat.csv', index=False)

In [ ]:
# good for submit
submission_lgb = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": y_pred_lgb
    })
submission_lgb.to_csv('submission_lgb.csv', index=False)

# 6.5 Funny Combine

In [ ]:
submission_rfc_cat = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": (y_pred_rfc +y_pred_cat)/2
    })
submission_rfc_cat.to_csv('submission_rfc_cat.csv', index=False)

In [ ]:
submission_lgb_cat = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": (y_pred_lgb +y_pred_cat)/2
    })
submission_lgb_cat.to_csv('submission_lgb_cat.csv', index=False)

In [ ]:
submission_rfc_lgb = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": (y_pred_rfc +y_pred_lgb)/2
    })
submission_rfc_lgb.to_csv('submission_rfc_lgb.csv', index=False)